# Split and Create Datasets

# To-Do
- Existing Data Sources
    - CCD Unlabelled
    - CCD Labeled
    - Synthetic Data (2000 mult-turn dialog)
<br><br>
- FROM Synthetic Data,  SPLIT via <u>stratified sampling</u>
    - Synth_data_1500_train_validate
    - Synth_data_250_test
    - Synth_data_250_simulation
<br><br>
- THEN create the following datasets
    - For <b>Topic Modeling</b>
        - Dataset Name: <u><i>CCD_Unlabeled_Final</i></u>
            - single customer utterances, not multi-turn dialog
            - CCD Unlabelled + Synth_data_1500_train_validate (<b>remove labels + data prep</b>) + Synth_data_250_test (<b>remove labels + data prep</b>)<p></p>
    - For <b>ABSA Final Classifier training and validation</b> (Pdt clf, action clf, sentiment clf)
        1. Dataset Name: <u><i>CCD_Labeled_Train_Validate</i></u>
            - single customer utterances, not multi-turn dialog
            - CCD Labeled + + Synth_data_1500_train_validate (<b>data prep, keep labels</b>)
        2. Dataset Name: <u><i>CCD_Labeled_Test</i></u>
            - single customer utterances, not multi-turn dialog
            - Synth_data_250_test (<b>data prep, keep labels</b>)
        - NB: topic modeling also uses this sentiment classifier<p></p>
    - For <b>Simulation of usage of Dashboard</b>
        - Dataset Name: <u><i>Simulated_CC_Dialog_Data</i></u>
            - multi-turn dialog, includes both Customer and CSE utterances
            - Synth_data_250_simulation
                - No data prep, No remove labels, it is just stratified split from "Synthetic Data (2000 mult-turn dialog)"<p></p>

# 1.0 Load and Split Synthetic Data
- Synth_data_250_simulation
- Synth_data_250_test
- Synth_data_1500_train_validate

In [1]:
import duckdb
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_parquet("input_data/Generated_dialogs.parquet")
print(df.shape)
print(df['dlg_id'].nunique())
display(df.head(3))

(14774, 7)
1995


dlg_id  turn                     speaker  \
0  2024-10-14__dlg_0     0  Customer Service Executive   
1  2024-10-14__dlg_0     1                    Customer   
2  2024-10-14__dlg_0     2  Customer Service Executive   

                                                text overall_dlg_pdt  \
0  Hello, thank you for calling [Bank Name]. My n...             atm   
1  Hi Alex! I'm doing great, thanks for asking. I...             atm   
2  I'm glad to hear you're excited about your new...             atm   

  overall_dlg_action overall_dlg_sentiment  
0               find         very positive  
1               find         very positive  
2               find         very positive

In [3]:
sql_query = """
select overall_dlg_pdt, dlg_id, turn
, ANY_VALUE(speaker) as speaker
, ANY_VALUE(overall_dlg_action) as overall_dlg_action
, ANY_VALUE(overall_dlg_sentiment) as overall_dlg_sentiment
, ANY_VALUE(text) as text
from df
group by overall_dlg_pdt, dlg_id, turn
order by overall_dlg_pdt, dlg_id, turn;
"""
df = duckdb.query(sql_query).to_df()
print(df.shape)
print(df['dlg_id'].nunique())
display(df.head(3))
display(df.tail(3))

# if this works, I can just use skip row sampling to reach estd 1500, 250, 250 dialogs

(14774, 7)
1995


overall_dlg_pdt               dlg_id  turn                     speaker  \
0         account  2024-10-14__dlg_101     0  Customer Service Executive   
1         account  2024-10-14__dlg_101     1                    Customer   
2         account  2024-10-14__dlg_101     2  Customer Service Executive   

  overall_dlg_action overall_dlg_sentiment  \
0               link              positive   
1               link              positive   
2               link              positive   

                                                text  
0  Good morning! Thank you for calling [Bank Name...  
1  Hi Alex! I'm really happy with the service I'v...  
2  I'm glad to hear that you've had a positive ex...

overall_dlg_pdt               dlg_id  turn                     speaker  \
14771         website  2024-10-14__dlg_992     4  Customer Service Executive   
14772         website  2024-10-14__dlg_992     5                    Customer   
14773         website  2024-10-14__dlg_992     6  Customer Service Executive   

      overall_dlg_action overall_dlg_sentiment  \
14771              block         very positive   
14772              block         very positive   
14773              block         very positive   

                                                    text  
14771  Thank you for that information. We can help yo...  
14772  That would be perfect. Please go ahead and set...  
14773  You're welcome! I've set the block on transact...

### 1.1 Get Indices of each split
- Synth_data_250_simulation
- Synth_data_250_test
- Synth_data_1500_train_validate

In [4]:
df.dlg_id.nunique(), df.shape[0]

(1995, 14774)

In [5]:
synth_data_250_simulation_indices = set()
synth_data_250_test_indices = set()
synth_data_1500_train_validate_indices = set()

total_dialogs = df.dlg_id.nunique()
all_dlg_ids = df.dlg_id.unique()

turn = 1
for i in range(total_dialogs):
    if turn > 10:
        turn = 1
    
    if turn == 1:
        synth_data_250_simulation_indices.add(all_dlg_ids[i])
        turn += 1
    elif turn ==  2:        
        synth_data_250_test_indices.add(all_dlg_ids[i])
        turn += 1
    elif turn <= 10:
        synth_data_1500_train_validate_indices.add(all_dlg_ids[i])
        turn += 1
       
print(len(synth_data_250_simulation_indices))
print(len(synth_data_250_test_indices))
print(len(synth_data_1500_train_validate_indices))

200
200
1595


### 1.2 Dataset: synth_data_250_simulation
- when simulation data is ingested during dashboard usage, ensure that data prep is done for ABSA e.g. only customer utterance, product: fixed_deposit etc

In [6]:
synth_data_250_simulation_df = df[df['dlg_id'].isin(synth_data_250_simulation_indices)].copy().reset_index(drop=True)
print(synth_data_250_simulation_df.shape)
print(synth_data_250_simulation_df['dlg_id'].nunique())
display(synth_data_250_simulation_df.head(1))
display(synth_data_250_simulation_df.tail(1))

fc = synth_data_250_simulation_df['dlg_id'] == '2024-10-14__dlg_101'
fc = synth_data_250_simulation_df['dlg_id'] == '2024-10-14__dlg_958'

for row in synth_data_250_simulation_df.loc[fc, :].itertuples():
    print(f"turn: {row.turn} - {row.speaker}\n{row.text}\n")

synth_data_250_simulation_df.reset_index(drop=True, inplace=True)
synth_data_250_simulation_df.to_csv('output_data/synth_data_250_simulation.csv', index=False, encoding='utf-8')
synth_data_250_simulation_df.to_parquet('output_data/synth_data_250_simulation.parquet', index=False)

(1476, 7)
200


overall_dlg_pdt               dlg_id  turn                     speaker  \
0         account  2024-10-14__dlg_101     0  Customer Service Executive   

  overall_dlg_action overall_dlg_sentiment  \
0               link              positive   

                                                text  
0  Good morning! Thank you for calling [Bank Name...

overall_dlg_pdt               dlg_id  turn                     speaker  \
1475         website  2024-10-14__dlg_958     5  Customer Service Executive   

     overall_dlg_action overall_dlg_sentiment  \
1475             select         very negative   

                                                   text  
1475  I completely understand your concerns, and I'm...

turn: 0 - Customer
Hi, I'm really frustrated with the service I've been receiving from your bank's website. It's been incredibly slow and keeps crashing whenever I try to access my account. What is happening here?

turn: 1 - Customer Service Executive
Hello! I'm sorry to hear about the difficulties you've been facing with our website. I understand how frustrating that can be. Could you please provide more details about what time these issues occur and which features are particularly problematic? I'll do my best to help resolve these issues.

turn: 2 - Customer
The problems have been happening consistently over the past week, especially when I try to check my account balance and make online payments. It feels like your site just isn't reliable anymore.

turn: 3 - Customer Service Executive
Thank you for sharing the details. We've been aware of some technical issues affecting a part of our user base, and our IT team is actively working to address them. To assist you better, could you try

### 1.3 Dataset: synth_data_250_test

In [7]:
synth_data_250_test_df = df[df['dlg_id'].isin(synth_data_250_test_indices)].copy().reset_index(drop=True)
print(synth_data_250_test_df.shape)
print(synth_data_250_test_df['dlg_id'].nunique())
display(synth_data_250_test_df.head(1))
display(synth_data_250_test_df.tail(1))

# fc = synth_data_250_test_df['dlg_id'] == '2024-10-14__dlg_103'
fc = synth_data_250_test_df['dlg_id'] == '2024-10-14__dlg_96'

for row in synth_data_250_test_df.loc[fc, :].itertuples():
    print(f"turn: {row.turn} - {row.speaker}\n{row.text}\n")

synth_data_250_test_df.reset_index(drop=True, inplace=True)
synth_data_250_test_df.to_csv('output_data/synth_data_250_test.csv', index=False, encoding='utf-8')
synth_data_250_test_df.to_parquet('output_data/synth_data_250_test.parquet', index=False)

(1503, 7)
200


overall_dlg_pdt               dlg_id  turn   speaker overall_dlg_action  \
0         account  2024-10-14__dlg_103     0  Customer           transfer   

  overall_dlg_sentiment                                               text  
0              negative  Hi, I’m really frustrated with my current expe...

overall_dlg_pdt              dlg_id  turn                     speaker  \
1502         website  2024-10-14__dlg_96     5  Customer Service Executive   

     overall_dlg_action overall_dlg_sentiment  \
1502               find               neutral   

                                                   text  
1502  Absolutely, once you log in with your credenti...

turn: 0 - Customer
Hi, I'm trying to visit your bank's website, but I'm not quite sure if I have the correct URL. Could you help me with that?

turn: 1 - Customer Service Executive
Of course! I'm here to help. Our official bank website is www.ourbank.com. Is that the URL you were trying to access?

turn: 2 - Customer
I was trying something similar, but it seems like I might have made a typo. Could you confirm that this is the site where I can manage my accounts online?

turn: 3 - Customer Service Executive
Yes, that's correct. Once you visit www.ourbank.com, you can log in to your online banking account to manage your finances. Is there anything in particular you would like to do on our website today?

turn: 4 - Customer
I just wanted to check my recent transactions and make sure everything looks alright with my account. So, I can do that at www.ourbank.com, right?

turn: 5 - Customer Service Executive
Absolutely, once you log in with your credentials, you'll be able to view your trans

### 1.4 Dataset: Synth_data_1500_train_validate

In [8]:
synth_data_1500_train_validate_df = df[df['dlg_id'].isin(synth_data_1500_train_validate_indices)].copy().reset_index(drop=True)
print(synth_data_1500_train_validate_df.shape)
print(synth_data_1500_train_validate_df['dlg_id'].nunique())
display(synth_data_1500_train_validate_df.head(1))
display(synth_data_1500_train_validate_df.tail(1))

# fc = synth_data_1500_train_validate_df['dlg_id'] == '2024-10-14__dlg_1045'
fc = synth_data_1500_train_validate_df['dlg_id'] == '2024-10-14__dlg_992'

for row in synth_data_1500_train_validate_df.loc[fc, :].itertuples():
    print(f"turn: {row.turn} - {row.speaker}\n{row.text}\n")

synth_data_1500_train_validate_df.reset_index(drop=True, inplace=True)
synth_data_1500_train_validate_df.to_csv('output_data/synth_data_1500_train_validate.csv', index=False, encoding='utf-8')
synth_data_1500_train_validate_df.to_parquet('output_data/synth_data_1500_train_validate.parquet', index=False)

(11795, 7)
1595


overall_dlg_pdt                dlg_id  turn                     speaker  \
0         account  2024-10-14__dlg_1045     0  Customer Service Executive   

  overall_dlg_action overall_dlg_sentiment  \
0            receive         very positive   

                                                text  
0  Hello, and thank you for calling [Bank Name]. ...

overall_dlg_pdt               dlg_id  turn                     speaker  \
11794         website  2024-10-14__dlg_992     6  Customer Service Executive   

      overall_dlg_action overall_dlg_sentiment  \
11794              block         very positive   

                                                    text  
11794  You're welcome! I've set the block on transact...

turn: 0 - Customer Service Executive
Hello, thank you for calling [Bank Name] customer service. My name is Alex. How can I assist you today?

turn: 1 - Customer
Hi Alex, I hope you're doing well. I'm very impressed with the services provided by the bank so far, especially your prompt customer support. However, I'm facing a minor issue with a specific website that keeps charging my credit card, and I'd like to block any further payments to it. Can you help me with that?

turn: 2 - Customer Service Executive
Thank you so much for the kind words. I'm glad to hear that you're satisfied with our services! Regarding the website charges, I can definitely assist you in blocking payments to that site. Could you please provide me with the name of the website and any details about recent charges?

turn: 3 - Customer
Sure, the website is [Website Name]. While the services are quite satisfactory, I decided not to continue my subscription for now. However, I saw another charge on my card yesterday.


# 2.0 Create Dataset: CCD_Unlabeled_Final
- used by Topic Modelling
- single customer utterances, not multi-turn dialog
- CCD Unlabelled + synth_data_1500_train_validate (<b>**remove labels + data prep**</b>) + synth_data_250_test (<b>**remove labels + data prep**</b>)<p></p>

### 2.1 Load CCD Unlabelled

In [9]:
df_ccd_unlabeled = pd.read_parquet('input_data/CCD_Unlabeled.parquet')
print(df_ccd_unlabeled.shape)
display(df_ccd_unlabeled.head())

(7706, 3)


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text  
0                      i am still waiting on my card  
1  what can i do if my card still hasnt arrived a...  
2  i have been waiting over a week is the card st...  
3  can i track my card while it is in the process...  
4  how do i know if i will get my card or if it i...

### 2.3 Prep synth_data_250_test_df

In [10]:
# remove labels
# keep only customer utterance
dft_1 = synth_data_250_test_df.loc[:, ['dlg_id', 'speaker', 'text']].copy()
print(dft_1.shape)
fc1 = dft_1['speaker'] == 'Customer'
fc2 = dft_1['speaker'] == 'customer'
dft_1 = dft_1.loc[fc1 | fc2, :].copy()
print(dft_1.shape)
# add date
dft_1['date'] = '2024-10-05'
new_cols = ['date', 'dlg_id','text']  # dropped 'speaker' column
dft_1 = dft_1[new_cols].copy()
dft_1.reset_index(drop=True, inplace=True)
print(dft_1.shape)
display(dft_1.head())

(1503, 3)
(706, 3)
(706, 3)


date                dlg_id  \
0  2024-10-05   2024-10-14__dlg_103   
1  2024-10-05   2024-10-14__dlg_103   
2  2024-10-05   2024-10-14__dlg_103   
3  2024-10-05  2024-10-14__dlg_1134   
4  2024-10-05  2024-10-14__dlg_1134   

                                                text  
0  Hi, I’m really frustrated with my current expe...  
1  Honestly, I've been facing a lot of issues wit...  
2  I appreciate the offer, but this has been an o...  
3  Hi, Jordan! I'm Alice. I recently received my ...  
4  Certainly! There’s a transaction dated Septemb...

### 2.4 Prep synth_data_1500_train_validate

In [11]:
# remove labels
# keep only customer utterance
dft_2 = synth_data_1500_train_validate_df.loc[:, ['dlg_id', 'speaker', 'text']].copy()
print(dft_2.shape)
fc1 = dft_2['speaker'] == 'Customer'
fc2 = dft_2['speaker'] == 'customer'
dft_2 = dft_2.loc[fc1 | fc2, :].copy()
print(dft_2.shape)
# add date
dft_2['date'] = '2024-10-05'
new_cols = ['date', 'dlg_id','text']  # dropped 'speaker' column
dft_2 = dft_2[new_cols].copy()
dft_2.reset_index(drop=True, inplace=True)
print(dft_2.shape)
display(dft_2.tail())

(11795, 3)
(5562, 3)
(5562, 3)


date               dlg_id  \
5557  2024-10-05  2024-10-14__dlg_991   
5558  2024-10-05  2024-10-14__dlg_991   
5559  2024-10-05  2024-10-14__dlg_992   
5560  2024-10-05  2024-10-14__dlg_992   
5561  2024-10-05  2024-10-14__dlg_992   

                                                   text  
5557  Yes, I have. I’m logged in right now, but I’m ...  
5558  Thanks, Jordan. I found it. I’ll try going thr...  
5559  Hi Alex, I hope you're doing well. I'm very im...  
5560  Sure, the website is [Website Name]. While the...  
5561  That would be perfect. Please go ahead and set...

### 2.5 Create CCD_Unlabeled_Final

In [12]:
ccd_unlabeled_final = pd.DataFrame()
ccd_unlabeled_final = pd.concat([df_ccd_unlabeled, dft_1, dft_2])
ccd_unlabeled_final.reset_index(drop=True, inplace=True)
print(ccd_unlabeled_final.shape)
display(ccd_unlabeled_final.head(3))
display(ccd_unlabeled_final.tail(3))

ccd_unlabeled_final.to_csv('output_data/ccd_unlabeled_final.csv', index=False, encoding='utf-8')
ccd_unlabeled_final.to_parquet('output_data/ccd_unlabeled_final.parquet', index=False)

(13974, 3)


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   

                                                text  
0                      i am still waiting on my card  
1  what can i do if my card still hasnt arrived a...  
2  i have been waiting over a week is the card st...

date               dlg_id  \
13971  2024-10-05  2024-10-14__dlg_992   
13972  2024-10-05  2024-10-14__dlg_992   
13973  2024-10-05  2024-10-14__dlg_992   

                                                    text  
13971  Hi Alex, I hope you're doing well. I'm very im...  
13972  Sure, the website is [Website Name]. While the...  
13973  That would be perfect. Please go ahead and set...

In [13]:
del df_ccd_unlabeled
del dft_1
del dft_2

# 3.0 Create Datasets for training the ABSA Classifiers (Labeled)
- dataset: *CCD_Labeled_Train_Validate*
- dataset: *CCD_Labeled_Test*

In [14]:
def convert_double_underscore_to_space(item):
    tmp = item.split('__')
    if len(tmp) > 1:
        return " ".join(tmp)
    else:
        return tmp[0]

def convert_double_underscore_to_single_underscore(item):
    tmp = item.split('__')
    if len(tmp) > 1:
        return "_".join(tmp)
    else:
        return tmp[0]

def convert_single_underscore_to_space(item):
    tmp = item.split('_')
    if len(tmp) > 1:
        return " ".join(tmp)
    else:
        return tmp[0]

def convert_space_to_single_underscore(item):
    tmp = item.split()
    if len(tmp) > 1:
        return '_'.join(tmp)
    else:
        return tmp[0]

convert_space_to_single_underscore('fixed_deposit')

'fixed_deposit'

### !!!!!!!!!!!! Data Prep !!!!!!!!!!!!!!!!
- sorted(df['overall_dlg_pdt'].unique())
- convert "fixed deposit" to "fixed_deposit"  # note: single underscore

### 3.1 Create dataset: CCD_Labeled_Train_Validate
- single customer utterances, not multi-turn dialog
- CCD Labeled + + Synth_data_1500_train_validate (<b>**remove labels + data prep**</b>)

##### 3.1.1 Load CCD Labeled

In [15]:
ccd_labeled = pd.read_parquet('input_data/ccd_labelled_adjusted.parquet')
print(ccd_labeled.shape)
display(ccd_labeled.head())

(7706, 6)


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   
3  can i track my card while it is in the process...    card  inquire   
4  how do i know if i will get my card or if it i...    card  inquire   

  sentiment  
0  negative  
1  negative  
2  negative  
3   neutral  
4   neutral

##### 3.1.2 Prepare synth_data_1500_train_validate_df

In [16]:
dft_2 = synth_data_1500_train_validate_df.loc[:, :].copy()
# print(dft_2.columns)
print(dft_2.shape)

fc1 = dft_2['speaker'] == 'Customer'
fc2 = dft_2['speaker'] == 'customer'
dft_2 = dft_2.loc[fc1 | fc2, :].copy()
# print(dft_2.columns)
print(dft_2.shape)

# add date & drop speaker, drop turn
dft_2['date'] = '2024-10-05'
new_cols = ['date', 'dlg_id','text', 'overall_dlg_pdt', 'overall_dlg_action', 'overall_dlg_sentiment']  # dropped 'speaker' column
dft_2 = dft_2[new_cols].copy()

dft_2.rename(columns={'overall_dlg_pdt': 'product', 'overall_dlg_action': 'action', 'overall_dlg_sentiment': 'sentiment'}, inplace=True)
dft_2.reset_index(drop=True, inplace=True)

# new addition
dft_2['product'] = dft_2['product'].apply(lambda x: convert_space_to_single_underscore(x))

print(dft_2.shape)
# print(dft_2.columns)
print(sorted(dft_2['product'].unique()))
display(dft_2.tail())

(11795, 7)
(5562, 7)
(5562, 6)
['account', 'atm', 'card', 'credit_card', 'current_account', 'debit_card', 'fixed_deposit', 'forex_account', 'loan', 'mobile_app', 'others', 'savings_account', 'website']


date               dlg_id  \
5557  2024-10-05  2024-10-14__dlg_991   
5558  2024-10-05  2024-10-14__dlg_991   
5559  2024-10-05  2024-10-14__dlg_992   
5560  2024-10-05  2024-10-14__dlg_992   
5561  2024-10-05  2024-10-14__dlg_992   

                                                   text  product    action  \
5557  Yes, I have. I’m logged in right now, but I’m ...  website  exchange   
5558  Thanks, Jordan. I found it. I’ll try going thr...  website  exchange   
5559  Hi Alex, I hope you're doing well. I'm very im...  website     block   
5560  Sure, the website is [Website Name]. While the...  website     block   
5561  That would be perfect. Please go ahead and set...  website     block   

          sentiment  
5557        neutral  
5558        neutral  
5559  very positive  
5560  very positive  
5561  very positive

##### 3.1.3 Dataset: CCD_Labeled_Train_Validate

In [17]:
ccd_labeled_train_validate = pd.DataFrame()

ccd_labeled_train_validate = pd.concat([ccd_labeled, dft_2])

ccd_labeled_train_validate.reset_index(drop=True, inplace=True)
print(ccd_labeled_train_validate.shape)
print(sorted(ccd_labeled_train_validate['product'].unique()))

display(ccd_labeled_train_validate.head(3))
display(ccd_labeled_train_validate.tail(3))

ccd_labeled_train_validate.to_csv('output_data/ccd_labeled_train_validate.csv', index=False, encoding='utf-8')
ccd_labeled_train_validate.to_parquet('output_data/ccd_labeled_train_validate.parquet', index=False)

(13268, 6)
['account', 'atm', 'card', 'credit_card', 'current_account', 'debit_card', 'fixed_deposit', 'forex_account', 'loan', 'mobile_app', 'others', 'savings_account', 'website']


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   

  sentiment  
0  negative  
1  negative  
2  negative

date               dlg_id  \
13265  2024-10-05  2024-10-14__dlg_992   
13266  2024-10-05  2024-10-14__dlg_992   
13267  2024-10-05  2024-10-14__dlg_992   

                                                    text  product action  \
13265  Hi Alex, I hope you're doing well. I'm very im...  website  block   
13266  Sure, the website is [Website Name]. While the...  website  block   
13267  That would be perfect. Please go ahead and set...  website  block   

           sentiment  
13265  very positive  
13266  very positive  
13267  very positive

In [18]:
del ccd_labeled
del dft_2
del ccd_labeled_train_validate

### 3.2 Create dataset: CCD_Labeled_Test
- single customer utterances, not multi-turn dialog
- Synth_data_250_test (<b>**data prep, keep labels**</b>)

##### 3.2.1 Prepare Synth_data_250_test_df

In [19]:
dft_2 = synth_data_250_test_df.loc[:, :].copy()
# print(dft_2.columns)
print(dft_2.shape)
print(dft_2['dlg_id'].nunique())

fc1 = dft_2['speaker'] == 'Customer'
fc2 = dft_2['speaker'] == 'customer'
dft_2 = dft_2.loc[fc1 | fc2, :].copy()
# print(dft_2.columns)
print(dft_2.shape)

# add date & drop speaker, drop turn
dft_2['date'] = '2024-10-05'
new_cols = ['date', 'dlg_id','text', 'overall_dlg_pdt', 'overall_dlg_action', 'overall_dlg_sentiment']  # dropped 'speaker' column
dft_2 = dft_2[new_cols].copy()

dft_2.rename(columns={'overall_dlg_pdt': 'product', 'overall_dlg_action': 'action', 'overall_dlg_sentiment': 'sentiment'}, inplace=True)
dft_2.reset_index(drop=True, inplace=True)

# new addition
dft_2['product'] = dft_2['product'].apply(lambda x: convert_space_to_single_underscore(x))

print(dft_2.shape)
# print(dft_2.columns)
display(dft_2.tail())

(1503, 7)
200
(706, 7)
(706, 6)


date               dlg_id  \
701  2024-10-05  2024-10-14__dlg_878   
702  2024-10-05  2024-10-14__dlg_878   
703  2024-10-05   2024-10-14__dlg_96   
704  2024-10-05   2024-10-14__dlg_96   
705  2024-10-05   2024-10-14__dlg_96   

                                                  text  product action  \
701  It says something about a “server connection t...  website    pay   
702  I didn’t think of using the mobile app. I’ll g...  website    pay   
703  Hi, I'm trying to visit your bank's website, b...  website   find   
704  I was trying something similar, but it seems l...  website   find   
705  I just wanted to check my recent transactions ...  website   find   

    sentiment  
701  negative  
702  negative  
703   neutral  
704   neutral  
705   neutral

##### 3.2.2 Save dataset: ccd_labeled_test

In [20]:
dft_2.to_csv('output_data/ccd_labeled_test.csv', index=False, encoding='utf-8')
dft_2.to_parquet('output_data/ccd_labeled_test.parquet', index=False)

print(sorted(dft_2['product'].unique()))

['account', 'atm', 'card', 'credit_card', 'current_account', 'debit_card', 'fixed_deposit', 'forex_account', 'loan', 'mobile_app', 'others', 'savings_account', 'website']


In [21]:
del dft_2

# 4.0 Create Dataset for simulation when using Dashboard

### 4.1 Dataset Name: *Simulated_CC_Dialog_Data*
- multi-turn dialog
- includes both Customer and CSE utterances
- Using Synth_data_250_simulation
- Data prep, No remove labels

In [22]:
dft_2 = synth_data_250_simulation_df.loc[:, :].copy()
# print(dft_2.columns)
print(dft_2.shape)
print(dft_2['dlg_id'].nunique())

# fc1 = dft_2['speaker'] == 'Customer'
# fc2 = dft_2['speaker'] == 'customer'
# dft_2 = dft_2.loc[fc1 | fc2, :].copy()
# # print(dft_2.columns)
# print(dft_2.shape)

# add date & drop speaker, drop turn
# dft_2['date'] = '2024-10-05'
# new_cols = ['date', 'dlg_id','text', 'overall_dlg_pdt', 'overall_dlg_action', 'overall_dlg_sentiment']  # dropped 'speaker' column
# dft_2 = dft_2[new_cols].copy()

# dft_2.rename(columns={'overall_dlg_pdt': 'product', 'overall_dlg_action': 'action', 'overall_dlg_sentiment': 'sentiment'}, inplace=True)
dft_2.reset_index(drop=True, inplace=True)

print(dft_2.shape)
# print(dft_2.columns)
display(dft_2.tail(6))

(1476, 7)
200
(1476, 7)


overall_dlg_pdt               dlg_id  turn                     speaker  \
1470         website  2024-10-14__dlg_958     0                    Customer   
1471         website  2024-10-14__dlg_958     1  Customer Service Executive   
1472         website  2024-10-14__dlg_958     2                    Customer   
1473         website  2024-10-14__dlg_958     3  Customer Service Executive   
1474         website  2024-10-14__dlg_958     4                    Customer   
1475         website  2024-10-14__dlg_958     5  Customer Service Executive   

     overall_dlg_action overall_dlg_sentiment  \
1470             select         very negative   
1471             select         very negative   
1472             select         very negative   
1473             select         very negative   
1474             select         very negative   
1475             select         very negative   

                                                   text  
1470  Hi, I'm really frustrated with the service I'v...  
1471  Hello! I'm sorry to hear about the difficultie...  
1472  The problems have been happening consistently ...  
1473  Thank you for sharing the details. We've been ...  
1474  I've tried that already, but there's been no i...  
1475  I completely understand your concerns, and I'm...

### 4.2 Save dataset: simulated_cc_dialog_data

In [23]:
dft_2.to_csv('output_data/simulated_cc_dialog_data.csv', index=False, encoding='utf-8')
dft_2.to_parquet('output_data/simulated_cc_dialog_data.parquet', index=False)
del dft_2